In [16]:
import util
client = util.LLMClient(provider='gemini')

In [17]:
BINARY_LABEL = "Basophil"
# Initial meta prompt for the first iteration
other_labels = [label for label in util.CLASSES if label != BINARY_LABEL]
other_labels_str = ", ".join(other_labels)
MEDICAL_CONCEPTS_MAPPING = {
    "Basophil": "Nucleus=Segmented, NC Ratio=Low, Granularity=Yes, Color=Blue/Black (dense)",
    "Eosinophil": "Nucleus=Segmented, NC Ratio=Low, Granularity=Yes, Color=Red",
    "Lymphocyte": "Nucleus=Unsegmented, NC Ratio=High, Granularity=No, Size=Small",
    "Monocyte": "Nucleus=Unsegmented, NC Ratio=Low, Granularity=No",
    "Neutrophil": "Nucleus=Segmented, NC Ratio=Low, Granularity=Yes, Color=Blue",
}


meta_init_prompt = f"""Give 50 distinct textual descriptions of pairs of visual discriminative features to identify whether the peripheral blood cell is a {BINARY_LABEL} or not. 
These are the following features an expert would look for: Cell Size, Cell Shape, Nucleus Shape, Nuclear-Cytoplasmic Ratio, Chromatin-Density, Cytoplasm-Vacuole, Cytoplasm-Texture, Cytoplasm-Color, Granule-Type, Granule-Color, Granularity
{MEDICAL_CONCEPTS_MAPPING[BINARY_LABEL]}
Only provide the output as Python code in the following format: prompts = list[tuple[negative: str, positive: str]]. Let's think step-by-step"""

In [18]:
print(client.get_llm_response(meta_init_prompt))

```python
prompts = [
    ("Cells are typically small, less than 12 micrometers.", "Cells are generally larger, often exceeding 12 micrometers."),
    ("The cell appears round or oval with a regular outline.", "The cell may exhibit an irregular or slightly distorted shape."),
    ("The nucleus is uniformly round and clearly defined.", "The nucleus is often obscured by granules or appears lobulated."),
    ("The nucleus occupies a large portion of the cell.", "The nucleus occupies a relatively small portion of the cell."),
    ("Chromatin appears finely dispersed and evenly distributed.", "Chromatin appears dense and condensed, often obscuring detail."),
    ("Cytoplasm lacks visible vacuoles.", "Cytoplasm contains prominent, often numerous, vacuoles."),
    ("Cytoplasm appears smooth and homogenous.", "Cytoplasm appears granular and textured."),
    ("Cytoplasm is pale pink or reddish.", "Cytoplasm is typically a bluish-grey or slightly lavender hue."),
    ("Granules are absent or ver